In [1]:
import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import doctable as dt

In [2]:
schema = (
    ('id','integer',dict(primary_key=True, autoincrement=True)),
    ('name','string', dict(nullable=False)),
    ('age','integer'),
    ('is_old', 'boolean'),
)
db = dt.DocTable2(schema, tabname='mydocuments', verbose=True)
# defaults: #fname=':memory:', engine='sqlite', persistent_conn=True, new_db=True
# fname=':memory:' is special - it loads database into memory
print(db)

DocTable2 Query: SELECT count() AS count_1 
FROM mydocuments
 LIMIT :param_1
<DocTable2::mydocuments ct: 0>


In [3]:
N = 10
for i in range(N):
    age = random.random() # number in [0,1]
    is_old = age > 0.5
    db.insert({'name':'user_'+str(i), 'age':age, 'is_old':is_old}, verbose=False)
print(db)

DocTable2 Query: SELECT count() AS count_1 
FROM mydocuments
 LIMIT :param_1
<DocTable2::mydocuments ct: 10>


## Notes on DB Interface
Just a few notes on the basic interface before looking at selects.

In [4]:
# subscript is used to access underlying sqlalchemy column reference (without querying data)
db['id']

Column('id', Integer(), table=<mydocuments>, primary_key=True, nullable=False)

In [5]:
# conditionals are applied directly to the column objects (as we'll see with "where" clause)
db['id'] < 3

In [6]:
# can also access using .col() method
db.col('id')

Column('id', Integer(), table=<mydocuments>, primary_key=True, nullable=False)

In [7]:
# to access all column objects (only useful for working directly with sql info)
db.columns

In [8]:
# to access more detailed schema information
db.schemainfo

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'age',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'is_old',
  'type': BOOLEAN(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

## Regular Selects
These functions all return lists of ResultProxy objects. As such, they can be accessed using numerical indices or keyword indices. For instance, if one select output row is ```row=(1, 'user_0')``` (after selecting "id" and "user"), it can be accessed such that ```row[0]==row['id']``` and ```row[1]==row['user']```.

In [9]:
# the limit argument means the result will only return some rows.
# I'll use it for convenience in these examples.
# this selects all rows
db.select(limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments
 LIMIT :param_1


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [10]:
db.select([db['id'],db['name']], limit=1)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name 
FROM mydocuments
 LIMIT :param_1


[(1, 'user_0')]

In [11]:
db.select_first()

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments
 LIMIT :param_1


(1, 'user_0', 0.3338617203704357, False)

In [12]:
db.select(db['name'],limit=5)

DocTable2 Query: SELECT mydocuments.name 
FROM mydocuments
 LIMIT :param_1


['user_0', 'user_1', 'user_2', 'user_3', 'user_4']

In [13]:
db.select_first(db['age'])

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments
 LIMIT :param_1


0.3338617203704357

## Conditional Selects

In [14]:
db.select(where=db['id']==2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id = :id_1


[(2, 'user_1', 0.6255094477138036, True)]

In [15]:
db.select(where=db['id']<3)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id < :id_1


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [16]:
# note parantheses to handle order of ops with overloaded bitwise ops
db.select(where= (db['id']>=2) & (db['id']<=4) & (db['name']!='user_2'))

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id >= :id_1 AND mydocuments.id <= :id_2 AND mydocuments.name != :name_1


[(2, 'user_1', 0.6255094477138036, True),
 (4, 'user_3', 0.9619765050962884, True)]

In [17]:
db.select(where=db['name'].in_(('user_2','user_3')))

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.name IN (:name_1, :name_2)


[(3, 'user_2', 0.11978049541417046, False),
 (4, 'user_3', 0.9619765050962884, True)]

In [18]:
db.select(where=db['id'].between(2,4))

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id BETWEEN :id_1 AND :id_2


[(2, 'user_1', 0.6255094477138036, True),
 (3, 'user_2', 0.11978049541417046, False),
 (4, 'user_3', 0.9619765050962884, True)]

In [19]:
# use of logical not operator "~"
db.select(where= ~(db['name'].in_(('user_2','user_3'))) & (db['id'] < 4))

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.name NOT IN (:name_1, :name_2) AND mydocuments.id < :id_1


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [20]:
# more verbose operators .and_, .or_, and .not_ are bound to the doctable package
db.select(where= dt.or_(dt.not_(db['id']==4)) & (db['id'] <= 2))

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id != :id_1 AND mydocuments.id <= :id_2


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [21]:
# now with simple computation
ages = db.select(db['age'])
mean_age = sum(ages)/len(ages)
db.select(db['name'], where=db['age']>mean_age, limit=2)

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments
DocTable2 Query: SELECT mydocuments.name 
FROM mydocuments 
WHERE mydocuments.age > :age_1
 LIMIT :param_1


['user_1', 'user_3']

In [22]:
# apply .label() method to columns
dict(db.select_first([db['age'].label('myage'), db['name'].label('myname')]))

DocTable2 Query: SELECT mydocuments.age AS myage, mydocuments.name AS myname 
FROM mydocuments
 LIMIT :param_1


{'myage': 0.3338617203704357, 'myname': 'user_0'}

## Column Operators
I bind the .min, .max, .count, .sum, and .mode methods to the column objects. Additionally, I move the .count method to a separate DocTable2 method.

In [23]:
db.select_first([db['age'].sum, db['age'].count, db['age']])

DocTable2 Query: SELECT sum(mydocuments.age) AS sum_1, count(mydocuments.age) AS count_1, mydocuments.age 
FROM mydocuments
 LIMIT :param_1


(5.135533845492732, 10, 0.3338617203704357)

In [24]:
# with labels now
dict(db.select_first([db['age'].sum.label('sum'), db['age'].count.label('ct')]))

DocTable2 Query: SELECT sum(mydocuments.age) AS sum, count(mydocuments.age) AS ct 
FROM mydocuments
 LIMIT :param_1


{'sum': 5.135533845492732, 'ct': 10}

## ORDER BY, GROUP BY, LIMIT
These additional arguments have also been provided.

In [25]:
# the limit is obvious - it has been used throughout these examples
db.select(limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments
 LIMIT :param_1


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [26]:
# orderby clause
db.select(orderby=db['age'].desc(), limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments ORDER BY mydocuments.age DESC
 LIMIT :param_1


[(9, 'user_8', 0.9823990399439722, True),
 (4, 'user_3', 0.9619765050962884, True)]

In [27]:
# compound orderby
db.select(orderby=(db['age'].desc(),db['is_old'].asc()), limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments ORDER BY mydocuments.age DESC, mydocuments.is_old ASC
 LIMIT :param_1


[(9, 'user_8', 0.9823990399439722, True),
 (4, 'user_3', 0.9619765050962884, True)]

In [28]:
# can also use column name directly
# can only use ascending and can use only one col
db.select(orderby='age', limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments ORDER BY mydocuments.age
 LIMIT :param_1


[(6, 'user_5', 0.0674230576929149, False),
 (3, 'user_2', 0.11978049541417046, False)]

In [29]:
# groupby clause
# returns first row of each group without any aggregation functions
db.select(groupby=db['is_old'])

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments GROUP BY mydocuments.is_old


[(1, 'user_0', 0.3338617203704357, False),
 (2, 'user_1', 0.6255094477138036, True)]

In [46]:
# compound groupby (weird example bc name is unique - have only one cat var in this demo)
db.select(groupby=(db['is_old'],db['name']), limit=3)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments GROUP BY mydocuments.is_old, mydocuments.name
 LIMIT :param_1


[(1, 'user_0', 0.3338617203704357, False),
 (3, 'user_2', 0.11978049541417046, False),
 (5, 'user_4', 0.23788732920923328, False)]

In [31]:
# groupby clause using max aggregation function
# gets match age for both old and young groups
db.select(db['age'].max, groupby=db['is_old'])

DocTable2 Query: SELECT max(mydocuments.age) AS max_1 
FROM mydocuments GROUP BY mydocuments.is_old


[0.335188247742367, 0.9823990399439722]

## SQL String Commands and Additional Clauses
For cases where DocTable2 does not provide a convenient interface, you may submit raw SQL commands. These may be a bit more unwieldly, but they offer maximum flexibility. They may be used either as simply an addition to the WHERE or arbitrary end clauses, or accessed in totality.

In [32]:
qstr = 'SELECT age,name FROM {} WHERE id=="{}"'.format(db.tabname, 1)
results = db.execute(qstr)
dict(list(results)[0])

DocTable2 Query: SELECT age,name FROM mydocuments WHERE id=="1"


{'age': 0.3338617203704357, 'name': 'user_0'}

In [33]:
whrstr = 'is_old=="{}"'.format('1')
db.select(whrstr=whrstr, limit=2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE is_old=="1"
 LIMIT :param_1


[(2, 'user_1', 0.6255094477138036, True),
 (4, 'user_3', 0.9619765050962884, True)]

In [34]:
# combine whrstr with structured query where clause
whrstr = 'is_old=="{}"'.format('1')
db.select(where=db['id']<=5, whrstr=whrstr)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments 
WHERE mydocuments.id <= :id_1 AND is_old=="1"


[(2, 'user_1', 0.6255094477138036, True),
 (4, 'user_3', 0.9619765050962884, True)]

## Count Method and Get Next ID
```.count()``` is a convenience method. Mostly the same could be accomplished by ```db.select_first(db['id'].count)```, but this requires no reference to a specific column.

```.next_id()``` is especially useful if one hopes to enter the id (or any primary key column) into new rows manually. Especially useful because SQL engines don't provide new ids except when a single insert is performed.

In [35]:
db.count()

DocTable2 Query: SELECT count() AS count_1 
FROM mydocuments
 LIMIT :param_1


10

In [36]:
db.count(db['age'] < 0.5)

DocTable2 Query: SELECT count() AS count_1 
FROM mydocuments 
WHERE mydocuments.age < :age_1
 LIMIT :param_1


5

In [37]:
db.next_id()

DocTable2 Query: SELECT max(mydocuments.id) AS max_1 
FROM mydocuments
 LIMIT :param_1


11

In [38]:
# weird (but possible) with 'age' because it's not an actual primary key
db.next_id(idcol='age')

DocTable2 Query: SELECT max(mydocuments.age) AS max_1 
FROM mydocuments
 LIMIT :param_1


1.9823990399439722

## Select as Pandas Series and DataFrame
These are especially useful when working with metadata because Pandas provides robust descriptive and plotting features than SQL alone. Good for generating sample information.

In [39]:
# must provide only a single column
db.select_series(db['age']).head(2)

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments


0    0.333862
1    0.625509
dtype: float64

In [40]:
db.select_series(db['age']).quantile([0.025, 0.985])

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments


0.025    0.079203
0.985    0.979642
dtype: float64

In [41]:
db.select_df([db['id'],db['age']]).head(2)

DocTable2 Query: SELECT mydocuments.id, mydocuments.age 
FROM mydocuments


,id,age
0,1,0.333862
1,2,0.625509


In [42]:
# must provide list of cols (even for one col)
db.select_df([db['id'],db['age']]).corr()

DocTable2 Query: SELECT mydocuments.id, mydocuments.age 
FROM mydocuments


,id,age
id,1.000000,0.416978
age,0.416978,1.000000


In [43]:
db.select_df([db['id'],db['age']]).describe().T

DocTable2 Query: SELECT mydocuments.id, mydocuments.age 
FROM mydocuments


,count,mean,std,min,25%,50%,75%,max
id,10.0,5.500000,3.027650,1.000000,3.250000,5.500000,7.750000,10.000000
age,10.0,0.513553,0.353842,0.067423,0.261881,0.423405,0.876292,0.982399


In [44]:
mean_age = db.select_series(db['age']).mean()
df = db.select_df([db['id'],db['age']])
df['old_grp'] = df['age'] > mean_age
df.groupby('old_grp').describe()

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments
DocTable2 Query: SELECT mydocuments.id, mydocuments.age 
FROM mydocuments


id                                              age            \
        count  mean       std  min  25%  50%   75%   max count      mean   
old_grp                                                                    
False     6.0  5.00  2.607681  1.0  3.5  5.5  6.75   8.0   6.0  0.267627   
True      4.0  6.25  3.862210  2.0  3.5  6.5  9.25  10.0   4.0  0.882443   

                                                                     
              std       min       25%       50%       75%       max  
old_grp                                                              
False    0.162055  0.067423  0.149307  0.285875  0.334857  0.511622  
True     0.171590  0.625509  0.876292  0.960931  0.967082  0.982399

In [45]:
# more complicated groupby aggregation.
# calculates the variance both for entries above and below average age
mean_age = db.select_series(db['age']).mean()
df = db.select_df([db['name'],db['age']])
df['old_grp'] = df['age']>mean_age
df.groupby('old_grp').agg(**{
    'first_name':pd.NamedAgg(column='name', aggfunc='first'),
    'var_age':pd.NamedAgg(column='age', aggfunc=np.var),
})

DocTable2 Query: SELECT mydocuments.age 
FROM mydocuments
DocTable2 Query: SELECT mydocuments.name, mydocuments.age 
FROM mydocuments


,first_name,var_age
old_grp,,
False,user_0,0.026262
True,user_1,0.029443
